In [2]:
import skipatom as sa
import pandas as pd
from skipatom import SkipAtomInducedModel
from skipatom import RandomVectors
import torch
import numpy as np
import random

In [3]:
input_data = pd.read_csv('./atomic_binding_energies.csv')

In [4]:
model_atoms = SkipAtomInducedModel.load(
    "embedding_data/mp_2020_10_09.dim200.model", 
    "embedding_data/mp_2020_10_09.training.data", 
    min_count=2e7, top_n=5)

# Get the unique orbitals
unique_orbitals = list(np.unique(input_data['Orbital']))

In [5]:
# Function to generate random vectors (for example, 3D vectors)
def generate_random_vector(dim=1):
    return [random.uniform(0, 1) for _ in range(dim)]

# Assign random vectors to each string in the list
orbital_vectors = {orbital: generate_random_vector() for orbital in unique_orbitals}


In [6]:
# Print all keys in model.dictionary
print(model_atoms.dictionary.keys())


dict_keys(['Si', 'C', 'Pb', 'I', 'Br', 'Cl', 'Eu', 'O', 'Fe', 'Sb', 'In', 'S', 'N', 'U', 'Mn', 'Lu', 'Se', 'Tl', 'Hf', 'Ir', 'Ca', 'Ta', 'Cr', 'K', 'Pm', 'Mg', 'Zn', 'Cu', 'Sn', 'Ti', 'B', 'W', 'P', 'H', 'Pd', 'As', 'Co', 'Np', 'Tc', 'Hg', 'Pu', 'Al', 'Tm', 'Tb', 'Ho', 'Nb', 'Ge', 'Zr', 'Cd', 'V', 'Sr', 'Ni', 'Rh', 'Th', 'Na', 'Ru', 'La', 'Re', 'Y', 'Er', 'Ce', 'Pt', 'Ga', 'Li', 'Cs', 'F', 'Ba', 'Te', 'Mo', 'Gd', 'Pr', 'Bi', 'Sc', 'Ag', 'Rb', 'Dy', 'Yb', 'Nd', 'Au', 'Os', 'Pa', 'Sm', 'Be', 'Ac', 'Xe', 'Kr'])


In [11]:
embeddings = []

for i in range(len(input_data['Atoms'])):
    try:
        atom_vector = torch.tensor(model_atoms.vectors[model_atoms.dictionary[input_data['Atoms'][i]]], dtype=torch.float32)
    except:
        atom_vector = torch.tensor(np.zeros(200), dtype=torch.float32)
        print(input_data['Atoms'][i])
    orbital_vector = torch.tensor(orbital_vectors[input_data['Orbital'][i]], dtype=torch.float32)
    final_embedding = torch.cat((atom_vector, orbital_vector))
    embeddings.append(np.array(final_embedding))


Ne
Ar
Ne
Ar


/var/folders/yy/r2ql1k410r7b95r_qpb4t3980000gn/T/ipykernel_23162/1818184346.py:11: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  embeddings.append(np.array(final_embedding))


In [12]:
embd = {'Atoms': input_data['Atoms'], 'Orbital': input_data['Orbital'], 'Embeddings': embeddings}
embd = pd.DataFrame(embd)
embd.to_csv('final_embedding.csv', index=False, header = True)